# N-grams

## Import librairies 

In [119]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/akaagi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [120]:
import math
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
pd.set_option('display.max_colwidth', None)
import re

## Load dataset

In [121]:
# df = pd.read_csv("../../../data_raw/kaggle_poem_dataset.csv")
# df.drop(columns=["Unnamed: 0", "Author", "Title", "Poetry Foundation ID"], inplace=True)
# df.rename(columns={"Content": "poem"}, inplace=True)
df = pd.read_csv("../../../data_raw/our_dataset.csv")
df.drop(columns=["Unnamed: 0", "topic"], inplace=True)
df.rename(columns={"Content": "poem"}, inplace=True)

# Define the remove_non_english function
def remove_non_english(text):
    # Regular expression to remove non-English alphabet characters
    english_only = re.sub(r'[^a-zA-Z\s.,;?!\'-]', '', text)
    return english_only

# Define the create_corpus function
def create_corpus(text):
    # Tokenize text into sentences
    sentences = text.split('\n')
    # Remove empty strings
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    # Join sentences into a corpus
    corpus = '\n'.join(sentences)  # Keep newline characters
    return corpus

# Apply remove_non_english function to poem column
df['poem'] = df['poem'].apply(remove_non_english)

# Apply create_corpus function to poem column
df['poem'] = df['poem'].apply(create_corpus)

# Remove \xa0
df['poem'] = df['poem'].str.replace('\xa0', '')

# Apply lowercase to the poem column
df['poem'] = df['poem'].str.lower()

df.tail(3)

,poem
10061,"i imagined the atmosphere would be clear,\nshot with pristine light,\nnot this sulphurous haze,\nthe air ionized as before a thunderstorm.\nmany have pictured a river here,\nbut no one mentioned all the boats,\ntheir benches crowded with naked passengers,\neach bent over a writing tablet.\ni knew i would not always be a child\nwith a model train and a model tunnel,\nand i knew i would not live forever,\njumping all day through the hoop of myself.\ni had heard about the journey to the other side\nand the clink of the final coin\nin the leather purse of the man holding the oar,\nbut how could anyone have guessed\nthat as soon as we arrived\nwe would be asked to describe this place\nand to include as much detail as possible\nnot just the water, he insists,\nrather the oily, fathomless, rat-happy water,\nnot simply the shackles, but the rusty,\niron, ankle-shredding shackles\nand that our next assignment would be\nto jot down, off the tops of our heads,\nour thoughts and feelings about being dead,\nnot really an assignment,\nthe man rotating the oar keeps telling us\nthink of it more as an exercise, he groans,\nthink of writing as a process,\na never-ending, infernal process,\nand now the boats have become jammed together,\nbow against stern, stern locked to bow,\nand not a thing is moving, only our diligent pens."
10062,"miss nancy ellicott\nstrode across the hills and broke them,\nrode across the hills and broke them\nthe barren new england hills\nriding to hounds\nover the cow-pasture.\nmiss nancy ellicott smoked\nand danced all the modern dances;\nand her aunts were not quite sure how they felt about it,\nbut they knew that it was modern.\nupon the glazen shelves kept watch\nmatthew and waldo, guardians of the faith,\nthe army of unalterable law."
10063,"in the dark we disappear, pure being.\nour mirror images, impure being.\nbeing and becoming heidegger, being and\nnothingness sartrewhich is purer being?\nbeing alone is no way to be thus\nloneliness is the test of pure being.\nnights in love i fell too far or not quite\nfar enoughone pure, one impure being.\nclouds, snow, mist, the dragon's breath on water,\nsmoke from firea metaphor's pure being.\nstillness and more stillness and the light locked\ndeep insideboth pure and impure being.\nis is the verb of being, i the noun\nor pronoun for the purists of being.\ni was, i am, i looked within and saw\nnothing very clearly purest being."


# Create corpus

In [122]:
corpus = " ".join(df['poem'])
corpus[:10]

'sunday we '

## Tokenization

In [123]:
corpus = corpus.lower()
tokens = word_tokenize(corpus)
print("Number of tokens in the training set:",len(tokens))

vocab = set(tokens)
print("Vocabulary size:",len(vocab))

Number of tokens in the training set: 3255376
Vocabulary size: 128002


## Train n-grams

In [124]:
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize

def tokenize(text):
    """Tokenize the input text."""
    return word_tokenize(text)

def count_ngrams(tokens, n):
    """Counts n-grams."""
    ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    return Counter(ngrams)

def calculate_ngram_probabilities(df, column, n, k=0.00001):
    """Calculates n-gram probabilities."""
    train_tokens = " ".join(df[column]).lower()
    train_tokens = tokenize(train_tokens)
    
    vocab = set(train_tokens)
    V = len(vocab)
    ngram_counts = count_ngrams(train_tokens, n)
    n_minus_one_gram_counts = count_ngrams(train_tokens, n-1)
    ngram_probabilities = defaultdict(float)
    
    for ngram in ngram_counts:
        prefix = ngram[:-1]
        ngram_counts[ngram] += k
        n_minus_one_gram_counts[prefix] += k
        ngram_probabilities[ngram] = (ngram_counts[ngram] + k) / (n_minus_one_gram_counts[prefix] + k*V)

    return ngram_probabilities

## Example usage

In [125]:
n = 5  # n-gram size
k = 0.00001  # Change this to the desired value for smoothing parameter k

ngram_probabilities = calculate_ngram_probabilities(df, 'poem', n, k)

In [126]:
print(f"Number of {n}-grams:",len(ngram_probabilities))

Number of 5-grams: 3139732


In [127]:
ngram_probabilities

defaultdict(float,
            {('sunday', 'we', 'lay', 'hands', 'on'): 0.43859949211194593,
             ('we', 'lay', 'hands', 'on', 'a'): 0.43859949211194593,
             ('lay', 'hands', 'on', 'a', 'girl'): 0.43859949211194593,
             ('hands', 'on', 'a', 'girl', 'of'): 0.43859949211194593,
             ('on', 'a', 'girl', 'of', 'ten'): 0.43859949211194593,
             ('a', 'girl', 'of', 'ten', 'hand'): 0.43859949211194593,
             ('girl', 'of', 'ten', 'hand', 'on'): 0.43859949211194593,
             ('of', 'ten', 'hand', 'on', 'hand'): 0.43859949211194593,
             ('ten', 'hand', 'on', 'hand', 'on'): 0.43859949211194593,
             ('hand', 'on', 'hand', 'on', 'cornsilk'): 0.43859949211194593,
             ('on', 'hand', 'on', 'cornsilk', 'hair'): 0.43859949211194593,
             ('hand', 'on', 'cornsilk', 'hair', '.'): 0.43859949211194593,
             ('on', 'cornsilk', 'hair', '.', 'we'): 0.43859949211194593,
             ('cornsilk', 'hair', '.', 'we', '

## Perplexity

In [128]:
def calculate_perplexity(test_tokens, ngram_probabilities, n):
    """Calculates the perplexity of a test corpus given n-gram probabilities."""
    log_probability_sum = 0
    ngram_count = 0
    
    for i in range(len(test_tokens)-n+1):
        ngram = tuple(test_tokens[i:i+n])
        log_probability_sum += math.log2(ngram_probabilities[ngram])
        ngram_count += 1
    
    average_log_probability = -log_probability_sum / ngram_count
    perplexity = math.pow(2, average_log_probability)
    
    return perplexity

In [129]:
calculate_perplexity(tokens, ngram_probabilities, n)

2.348875864416577

In [130]:
def greedy_sampling(context, vocab, ngram_probabilities, n, max_length = 50):
    
    sentence = []

    if len(context) < (n-1):
        print("len(context) < n")
        return sentence

    context = context[-(n-1):]
    
    for i in range(max_length):

        probs = dict()
        
        for v in vocab:

            ngram = list(context)
            ngram.append(v)
            ngram = tuple(ngram)
            probs[v] = ngram_probabilities[ngram]

        best_token = max(probs, key=probs.get) # greedy 
        #print(best_v)
        #print(probs[best_v])
        
        if probs[best_token] == 0:
            print("prob = 0")
            return sentence
            
        sentence.append(best_token)
        context = list(context)[1:]
        context.append(best_token)
        context = tuple(context)
            
    return sentence  

In [131]:
context = ['in', 'the', 'dark', 'we','disappear']
n_values = range(1, 7)  # Iterate n from 1 to 6
for n in n_values:
    print(f"{n}-gram :\n")
    k = 0.00001  # Smoothing parameter k
    
    # Calculate n-gram probabilities
    ngram_probabilities = calculate_ngram_probabilities(df, 'poem', n, k)
    
    # Generate sentence using greedy sampling
    sentence = greedy_sampling(context[:n], vocab, ngram_probabilities, n, max_length=200)
    
    # Print the context and generated sentence
    print("perplexity: " + f'{calculate_perplexity(tokens, ngram_probabilities, n)}')
    print(" ".join(context[:n]) + " " + " ".join(sentence))
    print("\n")


1-gram :

prob = 0
perplexity: 1180.0935812227215
in 


2-gram :

perplexity: 142.03690641192838
in the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the


3-gram :

perplexity: 13.323716982780782
in the dark , and the world , and the world , and the